## Importing required modules

In [1]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import base64
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [2]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
with open('token.json', 'rb') as token:
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('gmail', 'v1', credentials=creds)
profile = service.users().getProfile(userId='me').execute()
profile

{'emailAddress': 'dongoodygoody@gmail.com',
 'messagesTotal': 3806,
 'threadsTotal': 3074,
 'historyId': '2852648'}

In [3]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
with open('token.json', 'rb') as token:
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('gmail', 'v1', credentials=creds)

In [4]:
filter = service.users().messages().list(userId = 'me', maxResults=10,
                                         q = 'from:no_reply@accessbankplc.com \
                                             subject:AccessAlert Transaction Alert').execute()

filter_id = filter['messages']
id_lst = []
for ids in filter_id:
    id_lst.append(ids['id'])
    

In [6]:
trans_lst = []
for each_id in id_lst:
    msgs = service.users().messages().get(userId = 'me', id = each_id).execute()
    snippet = msgs['snippet']
    date = msgs['payload']['headers'][-1]['value']
    amount = re.search('\d*\.+\d+', snippet).group()
    acct_no = re.search('\d*\*+\d+', snippet).group()
    trans_lst.append({
        'amount': amount,
        'acct': acct_no,
        'datetime': date
    })
cols_name = ['amount', 'acct', 'datetime']
data = pd.DataFrame(trans_lst, columns=cols_name)

In [17]:
trans_lst = []
for each_id in id_lst:
    msgs = service.users().messages().get(userId = 'me', id = each_id).execute()
    snippet = msgs['snippet']
    main_body = msgs['payload']['body']['data']
    main_body = main_body.replace('-', '+').replace('_', '/')
    decode_msg = base64.b64decode(main_body)
    soup = bs(decode_msg, 'lxml')
    description_row = soup.find_all('tr')[7]
    
    description = description_row.find_all('td')[6].text.replace('\r',' ').replace("\n"," ").strip()
    date = msgs['payload']['headers'][-1]['value']
    amount = re.search('\d*\.+\d+', snippet).group()
    acct_no = re.search('\d*\*+\d+', snippet).group()
    
    
    trans_lst.append({
        'amount': amount,
        'acct': acct_no,
        'description': description,
        'datetime': date
    })
cols_name = ['amount', 'acct', 'description', 'datetime']
data = pd.DataFrame(trans_lst, columns=cols_name)

In [19]:
data

,amount,acct,description,datetime
0,200.00,162******608,AIRTIME/ MTN/08169327250,"Tue, 6 Sep 2022 17:25:10 +0530"
1,200.00,162******608,AIRTIME/ MTN/08169327250,"Mon, 5 Sep 2022 03:34:59 +0530"
2,5100.00,070******357,POS/WEB PMT LA 00NG,"Sun, 4 Sep 2022 22:35:43 +0530"
3,5000.00,070******357,TRF//FRM GOODRICH IFEANYI OKORO TO GOODRIC,"Sun, 4 Sep 2022 22:34:10 +0530"
4,5000.00,162******608,TRF//FRM GOODRICH IFEANYI OKORO TO GOODRIC,"Sun, 4 Sep 2022 22:34:09 +0530"
5,300.00,162******608,AIRTIME/ MTN/08168550974,"Sat, 3 Sep 2022 21:30:31 +0530"
6,300.00,070******357,AIRTIME/ MTN/08168550974,"Fri, 2 Sep 2022 16:04:05 +0530"
7,2100.00,070******357,POS/WEB PMT T_GOD IS ABLE ENTER 002392 207,"Thu, 1 Sep 2022 15:59:39 +0530"
8,5021.50,162******608,Amt includes COMM & VAT/USSD/GOODRICH IFEA,"Thu, 1 Sep 2022 15:17:34 +0530"
9,5000.00,070******357,Amt includes COMM & VAT/USSD/GOODRICH IFEA,"Thu, 1 Sep 2022 15:17:33 +0530"


In [9]:
d = soup.find_all('tr')[7]
e = d.find_all('td')[6].text.replace('\r',' ').replace("\n"," ").strip()
e

'Amt includes COMM & VAT/USSD/GOODRICH IFEA'

In [160]:
payload = msgs['payload']
headers = payload['headers']

parts = payload.get('body')['data']
parts = parts.replace('-','+').replace('_','/')
decode = base64.b64decode(parts)
soup = bs(decode,"lxml" )
body = soup.body()

[<center style="width: 100%; margin: 0; padding: 0; background-color: #f3f3f3;">
 <!--[if mso | IE]>
     <table role="presentation" border="0" cellpadding="0" cellspacing="0" width="100%" style="background-color: #f3f3f3;">
     <tr>
     <td>
     <![endif]-->
 <!--
             Set the email width. Defined in two places:
             1. max-width for all clients except Desktop Windows Outlook, allowing the email to squish on narrow but never go wider than 680px.
             2. MSO tags for Desktop Windows Outlook enforce a 680px width.
             Note: The Fluid and Responsive templates have a different width (600px). The hybrid grid is more "fragile", and I've found that 680px is a good width. Change with caution.
         -->
 <div class="email-container" style="max-width: 680px; margin: 0 auto;">
 <!--[if mso]>
             <table align="center" role="presentation" cellspacing="0" cellpadding="0" border="0" width="680">
             <tr>
             <td>
             <![endif

In [150]:
d = soup.find_all('tr')[7]
e = d.find_all('td')[6].text.replace('\r',' ').replace("\n"," ").strip()
e

'Amt includes COMM & VAT/USSD/GOODRICH IFEA'

In [ ]:
pat = re.compile(r'\d*\.?\d+')
for x in msg_lst:
   pad = re.findall('\d*\.?\d+', x)

print(pad[0])

11200.00


In [ ]:
import re
bank = []
pattern = re.compile(r'Debited')
b = you if re.search('Debited', you) else 3
print(b)

# import re
# pattern=re.compile(r'ack') 
# str_match = [x for x in my_list if re.search('ack', x)]
# print(str_match)

Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
